In [1]:
from gerrychain import Graph
import gurobipy as gp
from gurobipy import GRB
import geopandas as gpd
import networkx as nx

In [2]:
# Read Oklahoma county graph from the json file "OK_county.json"
filepath = "C:\\Users\\19186\\Downloads\\Alabama_Project\\"
filename = 'AL_tract.json'
filename_county = 'AL_county.json'

# GerryChain has a built-in function for reading graphs of this type:
G = Graph.from_json( filepath + filename )
CG = Graph.from_json( filepath + filename_county )

In [3]:
# For each node get its population
for node in G.nodes:
    G.nodes[node]['TOTPOP'] = G.nodes[node]['P0010001']

In [4]:
#Get Black Minorty and Voting Age Population
codes = ['P0030004'] # Black or African American alone 
codes += ['P0030011','P0030016','P0030017','P0030018','P0030019'] # Black or African American (among 2 races)
codes += ['P0030027','P0030028','P0030029','P0030030','P0030037','P0030038','P0030039','P0030040','P0030041','P0030042'] # among 3
codes += ['P0030048','P0030049','P0030050','P0030051','P0030052','P0030053','P0030058','P0030059','P0030060','P0030061'] # among 4
codes += ['P0030064','P0030065', 'P0030066','P0030067','P0030069'] # among 5
codes += ['P0030071'] # among 6

#Then, to sum the quantities associated with these codes, I do the following:
for i in G.nodes: 
        
# voting age population (VAP)
    G.nodes[i]['VAP'] = G.nodes[i]['P0030001']

 # Black voting age population (BVAP)
    G.nodes[i]['BVAP'] = sum(G.nodes[i][code] for code in codes )

In [5]:
#get the counties of each of the 
whole_counties = []
broken_counties = []
for node in CG.nodes:
    #We know Mobile and Jefferson County will have to be split to have 2 black majority districts
    #-- 097 mobile and -- 073 jefferson 
    if CG.nodes[node]['COUNTY'] in ['097','073']:
        broken_counties.append(node)
    else:
        whole_counties.append(node)

In [6]:
# Let's impose a 1% population deviation (+/- 0.5%)
deviation = 0.01

import math
k = 7          # number of districts
b = 2          # number of black majority districts
total_population = sum(G.nodes[node]['TOTPOP'] for node in G.nodes)

L = math.ceil((1-deviation/2)*total_population/k)
U = math.floor((1+deviation/2)*total_population/k)
print("Using L =",L,"and U =",U,"and k =",k)

Using L = 1249790 and U = 1262350 and k = 4


In [7]:
# create model 
m = gp.Model()

# create variables
x = m.addVars(G.nodes, k, vtype=GRB.BINARY) # x[i,j] equals one when tract i is assigned to district j
y = m.addVars(G.edges, vtype=GRB.BINARY)  # y[u,v] equals one when edge {u,v} is cut
z = m.addVars(CG.nodes, k, vtype=GRB.BINARY ) #some county c is assigned to district j
r = m.addVars(G.nodes, k, vtype=GRB.BINARY) # Add root variables: r[i,j] equals 1 if node i is the "root" of district j

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-18


In [8]:
# objective is to minimize weighted cut edges (district perimeter lengths)
m.setObjective( gp.quicksum(CG.edges[u,v]['shared_perim'] *y[u,v] for u,v in CG.edges ), GRB.MINIMIZE )

In [9]:
# add constraints saying that edge {i,j} is cut if i is assigned to district v but j is not.
m.addConstrs( z[u,j] - z[v,j] <= y[u,v] for u,v in CG.edges for j in range(k))

# county constraints
#keep these counties whole
m.addConstrs(gp.quicksum(z[c,j] for j in range(k)) == 1 for c in whole_counties)

#allow Mobile and Jefferson County to be split up to 2 times
m.addConstrs(gp.quicksum(z[c,j] for j in range(k)) <= 2 for c in broken_counties)

# add constraints saying that each tract i is assigned to one district
m.addConstrs( gp.quicksum(x[i,j] for j in range(k)) == 1 for i in G.nodes)

#add constraints to say tract i can only be apart of district j if their county c is
for i in G.nodes:
    for c in CG.nodes:
        if G.nodes[i]['COUNTY'] == CG.nodes[c]['COUNTY']:
            m.addConstrs(x[i,j] <= z[c,j] for j in range(k))

# add constraints saying that each district has population at least L and at most U
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes) >= L for j in range(k) )
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes) <= U for j in range(k) )

#add 2 black majority districts  
#ΣBi*xij >= .5*ΣWi*xij for all j in range(b)
m.addConstrs(gp.quicksum(G.nodes[i]['BVAP']* x[i,j] for i in G.nodes) >= (.5 * gp.quicksum(G.nodes[i]['VAP'] * x[i,j] for i in G.nodes)) for j in range(b))
 
m.update()

In [10]:
# We will use the contiguity constraints of Hojny et al. (MPC, 2021)
# Add flow variables: f[u,v] = amount of flow sent across arc uv 
# Flows are sent across arcs of the directed version of G which we call DG
DG = nx.DiGraph(CG)       #the directed version of CG
f = m.addVars( DG.edges ) #flow variables of county

In [11]:
# The big-M proposed by Hojny et al.
M = G.number_of_nodes() - k + 1

# Each district j should have one root
m.addConstrs(gp.quicksum(r[i,j] for i in CG.nodes) == 1 for j in range(k))

# If node i is not assigned to district j, then it cannot be its root
#do not need we since we set all roots
m.addConstrs(r[i,j] <= z[i,j] for i in CG.nodes for j in range(k))

# if not a root, consume some flow.
# if a root, only send out (so much) flow.
m.addConstrs(gp.quicksum( f[j,i] - f[i,j] for j in CG.neighbors(i) )>= 1 - M * gp.quicksum( r[i,j] for j in range(k) ) for i in CG.nodes)

# do not send flow across cut edges
m.addConstrs(f[i,j]+ f[j,i] <= M * (1 - y[i,j]) for i,j in CG.edges)

m.update()

In [ ]:
# solve IP model
m.optimize()

In [ ]:
print("The number of cut edges is",m.objval)

# retrieve the districts and their populations
districts = [ [i for i in G.nodes if x[i,j].x > 0.5] for j in range(k)]
district_populations = [ sum(G.nodes[i]["TOTPOP"] for i in districts[j]) for j in range(k) ]
black_voter_pop = [ sum(G.nodes[i]["BVAP"] for i in districts[j]) for j in range(k) ]
voter_pop = [ sum(G.nodes[i]["VAP"] for i in districts[j]) for j in range(k) ]



# print district info
for j in range(k):
    print("District",j,"has population",district_populations[j])
    print("it has a black voting age ratio", (black_pop[j] / white_pop[j]) * 100 , '%')
    print("")

In [15]:
# Let's draw it on a map
# Read Alabama tract shapefile from "AL_tract.shp"
filepath = 'C:\\Users\\19186\Downloads\\Alabama_Project\\'
filename = 'AL_tract.shp'

# Read geopandas dataframe from file
df = gpd.read_file( filepath + filename )

In [ ]:
# Which district is each county assigned to?
assignment = [ -1 for i in G.nodes ]

labeling = { i : j for i in G.nodes for j in range (k) if x[i,j].x > 0.5 }

# Now add the assignments to a column of the dataframe and map it
node_with_this_geoid = {G.nodes[i]['GEOID20'] : i for i in G.nodes }

# pick a position u in the dataframe
for u in range(G.number_of_nodes()):
    
    geoid = df['GEOID20'][u]
    
    # what node in G has this geoid?
    i = node_with_this_geoid[geoid]
    
    # postion u in the dataframe should be given
    # the same district # that county i has in 'labeling'
    assignment[u] = labeling[i]

#now add the assignents to a column of our dataframe then map it
df['assignment'] = assignment

my_fig = df.plot(column='assignment').get_figure()